<a href="https://colab.research.google.com/github/Prachii26/modern-ai-unsloth/blob/main/notebooks/Colab4_GRPO_Reasoning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --upgrade trl transformers

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-b32faur3/unsloth_386ea4a33dc8489c89d300fa7ca871f2
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-b32faur3/unsloth_386ea4a33dc8489c89d300fa7ca871f2
  Resolved https://github.com/unslothai/unsloth.git to commit 56392a7a4c1bf38154a144854a61b7d2d4dd2d0b
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.5/511.5 kB 45.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.7/276.7 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 113.1 MB/s eta 0:00:00

In [ ]:
from unsloth import FastLanguageModel
from datasets import load_dataset
from trl import GRPOTrainer, GRPOConfig
import torch

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "HuggingFaceTB/SmolLM2-135M-Instruct",
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True,
)

==((====))==  Unsloth 2025.11.1: Fast Llama patching. Transformers: 4.57.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/269M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/655 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

HuggingFaceTB/SmolLM2-135M-Instruct does not have a padding token! Will use pad_token = <|endoftext|>.


In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
)

Unsloth 2025.11.1 patched 30 layers with 30 QKV layers, 30 O layers and 30 MLP layers.


In [ ]:
dataset = load_dataset("openai/gsm8k", "main", split="train[:500]")
print(f"Total examples: {len(dataset)}")
print(dataset[0])

README.md: 0.00B [00:00, ?B/s]

main/train-00000-of-00001.parquet:   0%|          | 0.00/2.31M [00:00<?, ?B/s]

main/test-00000-of-00001.parquet:   0%|          | 0.00/419k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7473 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1319 [00:00<?, ? examples/s]

Total examples: 500
{'question': 'Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?', 'answer': 'Natalia sold 48/2 = <<48/2=24>>24 clips in May.\nNatalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May.\n#### 72'}


In [ ]:
def format_grpo(example):
    messages = [{"role": "user", "content": example["question"]}]
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    return {"prompt": prompt, "answer": example["answer"]}

dataset = dataset.map(format_grpo)
print(dataset[0])

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

{'question': 'Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?', 'answer': 'Natalia sold 48/2 = <<48/2=24>>24 clips in May.\nNatalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May.\n#### 72', 'prompt': '<|im_start|>system\nYou are a helpful AI assistant named SmolLM, trained by Hugging Face<|im_end|>\n<|im_start|>user\nNatalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?<|im_end|>\n<|im_start|>assistant\n'}


In [ ]:
def reward_function(prompts, completions, completion_ids, **kwargs):
    rewards = []
    for completion in completions:
        if any(char.isdigit() for char in completion):
            rewards.append(1.0)
        else:
            rewards.append(0.0)
    return rewards

In [ ]:
grpo_trainer = GRPOTrainer(
    model = model,
    reward_funcs = reward_function,
    args = GRPOConfig(
        per_device_train_batch_size = 8,
        gradient_accumulation_steps = 1,
        max_steps = 50,
        learning_rate = 5e-5,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 5,
        optim = "adamw_8bit",
        output_dir = "outputs",
    ),
    tokenizer = tokenizer,
    train_dataset = dataset,
)

In [ ]:
grpo_trainer.train()

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 500 | Num Epochs = 1 | Total steps = 50
O^O/ \_/ \    Batch size per device = 8 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (8 x 1 x 1) = 8
 "-____-"     Trainable parameters = 4,884,480 of 139,399,488 (3.50% trained)


Step,Training Loss,reward,reward_std,completions / mean_length,completions / min_length,completions / max_length,completions / clipped_ratio,completions / mean_terminated_length,completions / min_terminated_length,completions / max_terminated_length,sampling / sampling_logp_difference / mean,sampling / sampling_logp_difference / max,sampling / importance_sampling_ratio / min,sampling / importance_sampling_ratio / mean,sampling / importance_sampling_ratio / max,kl,rewards / reward_function / mean,rewards / reward_function / std
5,0.000000,1.000000,0.000000,208.525000,103.800000,256.000000,0.525000,161.460001,103.800000,229.600000,0,0,0,0,0,0.000000,1.000000,0.000000
10,0.000000,0.975000,0.070711,229.225000,173.600000,256.000000,0.650000,106.710004,71.200000,141.000000,No Log,No Log,No Log,No Log,No Log,0.000005,0.975000,0.070711
15,0.000000,0.975000,0.070711,225.250000,112.600000,256.000000,0.725000,153.083334,112.600000,198.400000,No Log,No Log,No Log,No Log,No Log,0.000028,0.975000,0.070711
20,0.000000,0.975000,0.070711,179.275000,45.600000,256.000000,0.450000,114.966667,45.600000,199.000000,No Log,No Log,No Log,No Log,No Log,0.000035,0.975000,0.070711
25,0.000000,0.975000,0.070711,194.775000,101.200000,238.800000,0.575000,143.158334,101.200000,190.800000,No Log,No Log,No Log,No Log,No Log,0.000055,0.975000,0.070711
30,0.000000,0.975000,0.070711,223.250000,140.200000,256.000000,0.675000,125.850000,89.000000,169.000000,No Log,No Log,No Log,No Log,No Log,0.000056,0.975000,0.070711
35,0.000000,0.950000,0.092582,202.075000,108.200000,240.400000,0.550000,161.416669,108.200000,204.600000,No Log,No Log,No Log,No Log,No Log,0.000069,0.950000,0.092582
40,0.000000,0.975000,0.070711,204.900000,100.800000,256.000000,0.550000,148.033337,100.800000,201.600000,No Log,No Log,No Log,No Log,No Log,0.000074,0.975000,0.070711
45,0.000000,1.000000,0.000000,205.300000,101.200000,256.000000,0.575000,110.250003,50.000000,174.000000,No Log,No Log,No Log,No Log,No Log,0.000068,1.000000,0.000000
50,0.000000,1.000000,0.000000,215.000000,116.800000,256.000000,0.625000,144.566669,116.800000,175.000000,No Log,No Log,No Log,No Log,No Log,0.000078,1.000000,0.000000


Unsloth: Will smartly offload gradients to save VRAM!


TrainOutput(global_step=50, training_loss=4.7136726948338034e-08, metrics={'train_runtime': 1092.7872, 'train_samples_per_second': 0.366, 'train_steps_per_second': 0.046, 'total_flos': 0.0, 'train_loss': 4.7136726948338034e-08})

In [ ]:
FastLanguageModel.for_inference(model)

messages = [{"role": "user", "content": "What is 15 + 27?"}]
inputs = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors="pt").to("cuda")
outputs = model.generate(input_ids=inputs, max_new_tokens=128, temperature=0.7)
print(tokenizer.decode(outputs[0]))

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


<|im_start|>system
You are a helpful AI assistant named SmolLM, trained by Hugging Face<|im_end|>
<|im_start|>user
What is 15 + 27?<|im_end|>
<|im_start|>assistant
To add 15 and 27, we simply add them together, which gives us 33.<|im_end|>


In [ ]:
model.save_pretrained("smollm2_grpo")
tokenizer.save_pretrained("smollm2_grpo")
print("✅ GRPO model saved!")

✅ GRPO model saved!
